<a href="https://colab.research.google.com/github/Victor1e/Project_Rumelhart_Romanian/blob/main/varianta_veche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Reshape, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random

# ==========================================
# 1. GENERARE DATE (Aceeasi ca la varianta Pro)
# ==========================================
def genereaza_verbe_regulate(n=1000):
    consoane = list("bcdfghjklmnprstvz")
    vocale = list("aeiou")
    verbe = []
    for _ in range(n):
        lungime = random.randint(3, 5)
        radacina = ""
        for i in range(lungime):
            if i % 2 == 0: radacina += random.choice(consoane)
            else: radacina += random.choice(vocale)

        if random.random() > 0.3:
            infinitiv = radacina + "a"
            trecut = radacina + "at"
        else:
            infinitiv = radacina + "i"
            trecut = radacina + "it"
        verbe.append((infinitiv, trecut))
    return verbe

verbe_reale = [
    ("fi", "fost"), ("avea", "avut"), ("bea", "baut"), ("da", "dat"),
    ("sta", "stat"), ("lua", "luat"), ("vedea", "vazut"), ("vrea", "vrut"),
    ("stii", "stiut"), ("veni", "venit"), ("scrie", "scris"), ("spune", "spus"),
    ("face", "facut"), ("zice", "zis"), ("duce", "dus"), ("rupe", "rupt"),
    ("coace", "copt"), ("fierbe", "fiert"), ("aduce", "adus"),
    ("manca", "mancat"), ("pleca", "plecat"), ("cauta", "cautat")
]

# Folosim acelasi dataset ca sa fie comparatia corecta
dataset = genereaza_verbe_regulate(3000) + verbe_reale * 50
random.shuffle(dataset)

# ==========================================
# 2. PREPROCESARE
# ==========================================
all_text = "".join([x[0] + x[1] for x in dataset])
chars = sorted(list(set(all_text)))
char_to_int = {c: i+2 for i, c in enumerate(chars)}
char_to_int['<PAD>'] = 0
char_to_int['<UNK>'] = 1
int_to_char = {i: c for c, i in char_to_int.items()}
vocab_size = len(char_to_int)
max_len = 16

def encode_word(word):
    return [char_to_int.get(c, 1) for c in word]

X = pad_sequences([encode_word(p[0]) for p in dataset], maxlen=max_len, padding='post')
Y = pad_sequences([encode_word(p[1]) for p in dataset], maxlen=max_len, padding='post')
Y_onehot = to_categorical(Y, num_classes=vocab_size)

X_train, X_test, y_train, y_test = train_test_split(X, Y_onehot, test_size=0.1, random_state=42)

# ==========================================
# 3. ARHITECTURA "RETRO" (1986 Style - MLP)
# ==========================================
model_retro = Sequential()

# Embedding (Pastram reprezentarea literelor)
model_retro.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len))

# --- DIFERENTA MAJORA AICI ---
# Aplatizam totul. Reteaua nu mai vede secventa (stanga-dreapta),
# ci vede o lista lunga de numere amestecate.
# E ca si cum ai incerca sa citesti o fraza vazand toate literele deodata, nu pe rand.
model_retro.add(Flatten())

# Hidden Layer (Pattern Associator)
# O retea densa clasica (Feed Forward)
model_retro.add(Dense(512, activation='relu'))
model_retro.add(Dropout(0.3))

# Output Layer
# Trebuie sa scoata tot cuvantul deodata
model_retro.add(Dense(max_len * vocab_size, activation='softmax'))
# Il rearanjam in forma de cuvant (Litere x Vocabular)
model_retro.add(Reshape((max_len, vocab_size)))

model_retro.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_retro.summary()

# ==========================================
# 4. ANTRENARE
# ==========================================
print("\n🕰️ Antrenam modelul VECHI (Retro)...")
history = model_retro.fit(X_train, y_train,
                    epochs=100,
                    batch_size=64,
                    validation_split=0.1,
                    verbose=1)

# ==========================================
# 5. TESTARE COMPARATIVA
# ==========================================
def predict_retro(word):
    vec = pad_sequences([encode_word(word)], maxlen=max_len, padding='post')
    pred = model_retro.predict(vec, verbose=0)
    res = ""
    for i in range(max_len):
        idx = np.argmax(pred[0][i])
        if idx > 1:
            res += int_to_char[idx]
    return res

print("\nREZULTATE MODEL VECHI")
teste = ["manca", "fi", "scrie", "boganiza", "programiza", "cauta", "vedea"]

for t in teste:
    res = predict_retro(t)
    print(f"Infinitiv: {t:12} -> Prezicere: {res}")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


🕰️ Antrenam modelul VECHI (Retro)...
Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6252 - loss: 1.7279 - val_accuracy: 0.8438 - val_loss: 0.6373
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8407 - loss: 0.5724 - val_accuracy: 0.9370 - val_loss: 0.3084
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9340 - loss: 0.2681 - val_accuracy: 0.9864 - val_loss: 0.1105
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9820 - loss: 0.1029 - val_accuracy: 0.9939 - val_loss: 0.0424
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9921 - loss: 0.0466 - val_accuracy: 0.9981 - val_loss: 0.0210
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9964 - loss: 0.0246 - val_accuracy: 0.9995 - val_loss: 0.0099
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9984 - loss: 0.0155 - val_accuracy: 1.0000 - val_loss: 0.0055
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.